## Loading data from youtube chanell

This project is based on Dataframe of videos from official youtube chanell of your president.

### Importing Required Libraries

To begin, we need to import the necessary libraries. We'll be using the `googleapiclient.discovery` library to access the YouTube Data API, the `pandas` library to create a DataFrame, and the `build` function provided by the `googleapiclient.discovery` library.

In [1]:
import googleapiclient.discovery
from googleapiclient.discovery import build
import googleapiclient.errors
import pandas as pd

### Retrieving Playlist Name

The `get_playlist_name` function retrieves the name of a YouTube playlist given its ID. It requires two parameters: `api_key`, which is your YouTube Data API key, and `playlist_id`, which is the ID of the playlist you want to retrieve.

In [2]:
def get_playlist_name(api_key, playlist_id):
    youtube = build('youtube', 'v3', developerKey=api_key)

    request = youtube.playlists().list(
        part="snippet",
        id=playlist_id
    )
    response = request.execute()

    if response.get("items"):
        playlist_name = response["items"][0]["snippet"]["title"]
        return playlist_name
    else:
        return None


### Retrieving Channel Videos

The `get_channel_videos` function retrieves all videos uploaded to a YouTube channel. It requires two parameters: `api_key`, which is your YouTube Data API key, and `channel_id`, which is the ID of the channel you want to retrieve videos from.

In [3]:
def get_channel_videos(api_key, channel_id):
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

    # Get the playlist ID of the channel's uploaded videos.
    channel_response = youtube.channels().list(
        part="contentDetails",
        id=channel_id
    ).execute()

    playlist_id = channel_response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Get all videos in the uploads playlist.
    videos = []
    ids=[]
    next_page_token = None

    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="snippet",
            maxResults=4000,
            playlistId=playlist_id,
            pageToken=next_page_token
        ).execute()
        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")
        if next_page_token is None:
            break

    return videos


### Creating Videos DataFrame

The `create_videos_dataframe` function creates a DataFrame from a list of YouTube videos. It requires one parameter: `videos`, which is a list of videos retrieved using the `get_channel_videos` function.

In [4]:
def create_videos_dataframe(videos):
    video_data = []

    for video in videos:
        video_data.append(video["snippet"])

    return pd.DataFrame(video_data)
